In [1]:
import pandas as pd
import json
import numpy as np
import pandas as pd
import os
from glob import glob
import re
import music21
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

from melody import Melody
from utils import get_chord_progressions
from objective_metrics import calculate_HC

In [2]:
version = '1.2'

In [3]:
info = pd.read_csv('../data/sources/WeimarDB/weimar_db_melody_info_with_pitch.csv')

In [4]:
res = pd.read_csv(f'../data/alignment_scores/v1.2/results-20-03-2022-20-05-48.csv', index_col=0).set_index('in_filename')

In [5]:
chord_progression = get_chord_progressions('..')

In [6]:
info

,performer,title,onset,pitch,bar,beat,tatum,subtatum,avgtempo,signature,chord,form,key,chord_changes
0,Art Pepper,Anthropology,9.171882,NaN,-1,1,NaN,NaN,218.8,4/4,NaN,I1,Bb-maj,A1: ||Bb6 G7 |C-7 F7 |Bb G-7 |C-7 F7 |F-7 Bb7 ...
1,Art Pepper,Anthropology,9.488254,NaN,-1,2,NaN,NaN,218.8,4/4,NaN,NaN,Bb-maj,A1: ||Bb6 G7 |C-7 F7 |Bb G-7 |C-7 F7 |F-7 Bb7 ...
2,Art Pepper,Anthropology,9.779955,NaN,-1,3,NaN,NaN,218.8,4/4,NaN,NaN,Bb-maj,A1: ||Bb6 G7 |C-7 F7 |Bb G-7 |C-7 F7 |F-7 Bb7 ...
3,Art Pepper,Anthropology,10.052608,NaN,-1,4,NaN,NaN,218.8,4/4,NaN,NaN,Bb-maj,A1: ||Bb6 G7 |C-7 F7 |Bb G-7 |C-7 F7 |F-7 Bb7 ...
4,Art Pepper,Anthropology,10.339796,65.0,0,1,1.0,0.0,218.8,4/4,Bb6,NaN,Bb-maj,A1: ||Bb6 G7 |C-7 F7 |Bb G-7 |C-7 F7 |F-7 Bb7 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235634,Zoot Sims,Night and Day,64.713311,NaN,55,2,NaN,NaN,217.6,4/4,NaN,NaN,D-maj,B1: ||Fj7 |Fj7 |Dj7 |Dj7 |Fj7 |Fj7 ...
235635,Zoot Sims,Night and Day,65.005896,NaN,55,3,NaN,NaN,217.6,4/4,NaN,NaN,D-maj,B1: ||Fj7 |Fj7 |Dj7 |Dj7 |Fj7 |Fj7 ...
235636,Zoot Sims,Night and Day,65.289524,NaN,55,4,NaN,NaN,217.6,4/4,NaN,NaN,D-maj,B1: ||Fj7 |Fj7 |Dj7 |Dj7 |Fj7 |Fj7 ...
235637,Zoot Sims,Night and Day,65.573152,NaN,56,1,NaN,NaN,217.6,4/4,NC,NaN,D-maj,B1: ||Fj7 |Fj7 |Dj7 |Dj7 |Fj7 |Fj7 ...


In [110]:
folder = f'../data/split_melody_data/v{version}/Weimar DB'

filepaths = [y for x in os.walk(folder) for y in glob(os.path.join(x[0], '*.csv'))]

In [111]:
filepaths

['../data/split_melody_data/v1.2/Weimar DB/Art Pepper - Anthropology -1-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Art Pepper - Anthropology -2-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Art Pepper - Anthropology -3-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Art Pepper - Anthropology -4-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Ben Webster - Bye Bye Blackbird -1-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Ben Webster - Bye Bye Blackbird -2-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Benny Carter - Just Friends -2-.csv',
 '../data/split_melody_data/v1.2/Weimar DB/Benny Carter - Just Friends -3-.csv',
 "../data/split_melody_data/v1.2/Weimar DB/Bob Berg - I Didn't Know What Time It Was -1-.csv",
 "../data/split_melody_data/v1.2/Weimar DB/Bob Berg - I Didn't Know What Time It Was -2-.csv",
 "../data/split_melody_data/v1.2/Weimar DB/Bob Berg - I Didn't Know What Time It Was -3-.csv",
 '../data/split_melody_data/v1.2/Weimar DB/Cannonball Adderley - So

In [112]:
file = filepaths[0]

In [ ]:
for file in filepaths:
    a = info[(info['performer'] == 'Bob Berg') & (info['title'] == 'I Didn\'t Know What Time It Was')]

    s = pd.read_csv(file, index_col=0)
    
    s['correct_chord'] = np.nan

    for i, row in s.iterrows():
        m = row['measure'] + 1
        o = int(np.floor(4 * row['offset'] / 48)) + 1

        q = a[(a['bar'] == m) & (a['beat'] == o)]
        chord = q['chord'].values[0]

    #     print(chord)

        while type(chord) == float:
            qi = int(q.index[0]) - 1
            q = a[a.index == qi]

            chord = q['chord'].values[0]

    #         print(chord)

    #     print('--')
    #     print(m, o, chord)
    #     print('----')

        s.loc[i, 'correct_chord'] = chord

In [113]:
file

'../data/split_melody_data/v1.2/Weimar DB/Art Pepper - Anthropology -1-.csv'

In [123]:
a = info[(info['performer'] == 'Bob Berg') & (info['title'] == 'I Didn\'t Know What Time It Was')]

In [124]:
a.head(20)

,performer,title,onset,bar,beat,avgtempo,signature,chord,form,key,chord_changes
7626,Bob Berg,I Didn't Know What Time It Was,2.960544,1,1,126.7,4/4,E-7,A1,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7627,Bob Berg,I Didn't Know What Time It Was,3.462132,1,2,126.7,4/4,NaN,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7628,Bob Berg,I Didn't Know What Time It Was,3.958912,1,3,126.7,4/4,A7,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7629,Bob Berg,I Didn't Know What Time It Was,4.447120,1,4,126.7,4/4,NaN,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7630,Bob Berg,I Didn't Know What Time It Was,4.918027,2,1,126.7,4/4,D-7,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7631,Bob Berg,I Didn't Know What Time It Was,5.416916,2,2,126.7,4/4,NaN,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7632,Bob Berg,I Didn't Know What Time It Was,5.895850,2,3,126.7,4/4,Bm7b5,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7633,Bob Berg,I Didn't Know What Time It Was,6.356825,2,4,126.7,4/4,NaN,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7634,Bob Berg,I Didn't Know What Time It Was,6.825941,3,1,126.7,4/4,E-7,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...
7635,Bob Berg,I Didn't Know What Time It Was,7.286848,3,2,126.7,4/4,NaN,NaN,F-maj,A1: ||E-7 A7 |D-7 Bm7b5 |E-7 A7 |D-7 G7 |G-7 C...


In [125]:
s = pd.read_csv(file, index_col=0)

In [126]:
s

,pitch,pitch_class,raw_ticks,quant_ticks,raw_duration,quant_duration,offset,measure,chord_name,chord_root,chord_bass,chord_notes,ticks,duration
4,63,Eb,0.243490,12,0.118490,6,12,0,Bbmaj7,Bb,Bb,"['A#', 'A#', 'D', 'F', 'A']",12,6
5,58,Bb,0.533854,26,0.171875,8,26,0,G7,G,G,"['G', 'G', 'B', 'D', 'F']",26,8
6,58,Bb,0.813802,39,0.438802,21,39,0,G7,G,G,"['G', 'G', 'B', 'D', 'F']",39,21
7,50,D,3.253906,156,0.144531,7,12,3,Cmin7,C,C,"['C', 'C', 'D#', 'G', 'A#']",156,7
8,57,A,3.493490,168,0.131510,6,24,3,F7,F,F,"['F', 'F', 'A', 'C', 'D#']",168,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,61,Db,31.425781,1508,0.078125,4,20,31,Bb6,Bb,Bb,"['A#', 'A#', 'D', 'F', 'G']",1508,4
123,62,D,31.523438,1513,0.114583,6,25,31,Bb6,Bb,Bb,"['A#', 'A#', 'D', 'F', 'G']",1513,6
124,58,Bb,31.647135,1519,0.105469,5,31,31,Bb6,Bb,Bb,"['A#', 'A#', 'D', 'F', 'G']",1519,5
125,56,Ab,31.768229,1525,0.119792,6,37,31,Bb6,Bb,Bb,"['A#', 'A#', 'D', 'F', 'G']",1525,6


In [127]:
s['correct_chord'] = np.nan

for i, row in s.iterrows():
    m = row['measure'] + 1
    o = int(np.floor(4 * row['offset'] / 48)) + 1
    
    q = a[(a['bar'] == m) & (a['beat'] == o)]
    chord = q['chord'].values[0]
    
#     print(chord)
    
    while type(chord) == float:
        qi = int(q.index[0]) - 1
        q = a[a.index == qi]
        
        chord = q['chord'].values[0]
        
#         print(chord)
  
#     print('--')
#     print(m, o, chord)
#     print('----')
    
    s.loc[i, 'correct_chord'] = chord

In [188]:
s[['measure', 'offset', 'chord_name', 'correct_chord']].head(10)

,measure,offset,chord_name,correct_chord
4,0,12,Bbmaj7,E-7
5,0,26,G7,A7
6,0,39,G7,A7
7,3,12,Cmin7,D-7
8,3,24,F7,G7
9,3,31,F7,G7
10,3,37,F7,G7
11,3,45,F7,G7
12,4,8,Fmin7,G-7
13,4,13,Fmin7,G-7


In [131]:
res.loc['Bob Berg - I Didn\'t Know What Time It Was.mid']

,cp_measures,n_measures,repetitions,min_measure,max_measure,outro_init,outro_auto,outro_difference,source,out_filename,starting_measure,melody_mido_key,chord_progression_key,transpose_semitones,harmonic_consistency,harmonic_consistency_mean
in_filename,,,,,,,,,,,,,,,,
Bob Berg - I Didn't Know What Time It Was.mid,36,110,3,2,109,0,1,-1,Weimar DB,Bob Berg - I Didn't Know What Time It Was -1-.mid,1,F major,G,2,"[1.0, 1.0, 1.0, 0.6666, 0.0, 1.0, 0.6666, 1.0,...",0.644424
Bob Berg - I Didn't Know What Time It Was.mid,36,110,3,2,109,0,1,-1,Weimar DB,Bob Berg - I Didn't Know What Time It Was -2-.mid,1,F major,G,2,"[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.6666, 1.0, 1....",0.680973
Bob Berg - I Didn't Know What Time It Was.mid,36,110,3,2,109,0,1,-1,Weimar DB,Bob Berg - I Didn't Know What Time It Was -3-.mid,1,F major,G,2,"[0.0, 0.0, 0.0, 0.6666, 0.0, 1.0, 0.0, 0.0, 1....",0.657002


In [141]:
info[info['form'] == 'A1'].drop_duplicates('melid', keep='first').sort_values('bar', ascending=False)

,melid,performer,title,onset,bar,beat,avgtempo,signature,chord,form,key,chord_changes
15381,59,Charlie Parker,Ko-Ko,54.465102,33,1,299.4,4/4,Bb6,A1,Bb-maj,B1: ||C#-7 |F#7 |Bj7 |Bj7 |B-7 |E7 ...
79834,271,Lee Konitz,Mean to Me,37.938503,17,1,138.9,4/4,Ebj7,A1,Eb-maj,B1: ||Abj7 |Bb7 Eb7 |Abj7 |Db7 C7 |F-7 |...
132171,456,Zoot Sims,Night and Day,22.462154,17,1,217.6,4/4,Em7b5,A1,D-maj,B1: ||Fj7 |Fj7 |Dj7 |Dj7 |Fj7 |Fj7 ...
84077,290,Lionel Hampton,Whispering,3.983741,1,1,99.5,4/4,Eb,A1,Eb-maj,A1: ||Eb |D7 |Eb |C+7 C7 |F7 |Bb7 |E...
84147,291,Louis Armstrong,Basin Street Blues,2.685714,1,1,107.2,4/4,Bb,A1,Bb-maj,A1: ||Bb |D7 |G7 |G7 |C7 |F7 |Bb B...
...,...,...,...,...,...,...,...,...,...,...,...,...
39284,150,Don Ellis,Sweet and Lovely,6.940998,1,1,134.1,4/4,G-7,A1,C-maj,A1: ||G-7 |C7 |G-7 |C7 |F7 |Bbj7 Eb7...
39148,149,Don Ellis,Out of Nowhere,7.069025,1,1,151.9,4/4,Gj7,A1,G-maj,A1: ||Gj7 |Gj7 |Bb-7 |Eb7 |Gj7 |Gj7 ...
38872,148,Don Ellis,Johnny Come Lately,4.721383,1,1,211.3,4/4,G-6,A1,G-min,A1: ||G-6 |Eb7 D7 |G-6 |Eb7 D7 |G-6 |Eb7...
38600,147,Don Ellis,I Love You,2.044898,1,1,146.9,NaN,Gm7b5,A1,F-maj,A1: ||Gm7b5 |C79b |Fj7 |Fj7 |G-7 |C7...


In [200]:
from utils import int_to_pitch

info['pitch_class'] = info['pitch'].apply(lambda x: int_to_pitch(int(x)) if x > 0 else np.nan)

In [215]:
info[info['title'] == 'Summertime'][['onset', 'pitch', 'pitch_class', 'form', 'bar', 'beat', 'tatum', 'chord', 'chord_changes']].head(40)

,onset,pitch,pitch_class,form,bar,beat,tatum,chord,chord_changes
0,1.534694,NaN,NaN,I1,-2,4,NaN,Am7b5,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
1,2.220249,NaN,NaN,NaN,-1,1,NaN,G-,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
2,2.828753,NaN,NaN,NaN,-1,2,NaN,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
3,3.492562,NaN,NaN,NaN,-1,3,NaN,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
4,4.184785,NaN,NaN,NaN,-1,4,NaN,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
5,4.874535,NaN,NaN,NaN,0,1,NaN,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
6,5.565556,66.0,F#4,NaN,0,2,1.0,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
7,5.565556,67.0,G4,NaN,0,2,2.0,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
8,5.565556,66.0,F#4,NaN,0,2,3.0,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
9,5.565556,67.0,G4,NaN,0,2,4.0,NaN,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...


In [213]:
info[info['title'] == 'Summertime']['form'].dropna()

0      I1
15     A1
56     B1
57     B1
58     B1
105    A1
106    A1
151    B1
218    A1
275    B1
276    B1
327    A1
384    B1
Name: form, dtype: object

In [203]:
res.loc['Sidney Bechet - Summertime.mid']

,cp_measures,n_measures,repetitions,min_measure,max_measure,outro_init,outro_auto,outro_difference,source,out_filename,starting_measure,melody_mido_key,chord_progression_key,transpose_semitones,harmonic_consistency,harmonic_consistency_mean
in_filename,,,,,,,,,,,,,,,,
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -1-.mid,2,B- major,Am,2,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.6666, 1.0, 1....",0.680169
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -2-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.6666, 1.0, 0.6666,...",0.666648
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -3-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666, 1.0, 0....",0.694764
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -4-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0.690039


In [179]:
chord_progression['Summertime']

{'sections': ['A'],
 'progression': {'A': ['Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'E7b13',
   'E7b13',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'A7b9',
   'A7b9',
   'Dmin7',
   'Dmin7',
   'Dmin7',
   'Dmin7',
   'F7',
   'F7',
   'F7',
   'F7',
   'B7#5',
   'B7#5',
   'B7#5',
   'B7#5',
   'E7b9',
   'E7b9',
   'E7b9',
   'E7b9',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'E7b13',
   'E7b13',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'D7',
   'D7',
   'G7b9',
   'G7b9',
   'Cmaj7',
   'Cmaj7',
   'Amin7',
   'Amin7',
   'Bm7b5',
   'Bm7b5',
   'E7b13',
   'E7b13',
   'Amin7',
   'Amin7',
   'Amin7',
   'Amin7',
   'Bm7b5',
   'Bm7b5',
   'E7b13',
   'E7b13']},
 'key': 'A',
 'minor': True,
 'time_signature': [4, 4]}

In [185]:
info_old = pd.read_csv('../data/sources/WeimarDB/weimar_db_melody_info.csv')

In [186]:
info_old[info_old['title'] == 'Summertime'].head(20)

,melid,performer,title,onset,bar,beat,avgtempo,signature,chord,form,key,chord_changes
109490,377,Sidney Bechet,Summertime,1.534694,-2,4,84.2,4/4,Am7b5,I1,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109491,377,Sidney Bechet,Summertime,2.220249,-1,1,84.2,4/4,G-,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109492,377,Sidney Bechet,Summertime,2.828753,-1,2,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109493,377,Sidney Bechet,Summertime,3.492562,-1,3,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109494,377,Sidney Bechet,Summertime,4.184785,-1,4,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109495,377,Sidney Bechet,Summertime,4.874535,0,1,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109496,377,Sidney Bechet,Summertime,5.565556,0,2,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109497,377,Sidney Bechet,Summertime,6.277166,0,3,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109498,377,Sidney Bechet,Summertime,7.013651,0,4,84.2,4/4,NaN,NaN,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...
109499,377,Sidney Bechet,Summertime,7.768549,1,1,84.2,4/4,G-,A1,G-min,A1: ||G- D7 |G- D7 |G- D7 |G- G7 |C- |C- |...


In [208]:
res.loc['Sidney Bechet - Summertime.mid']

,cp_measures,n_measures,repetitions,min_measure,max_measure,outro_init,outro_auto,outro_difference,source,out_filename,starting_measure,melody_mido_key,chord_progression_key,transpose_semitones,harmonic_consistency,harmonic_consistency_mean
in_filename,,,,,,,,,,,,,,,,
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -1-.mid,2,B- major,Am,2,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.6666, 1.0, 1....",0.680169
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -2-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.6666, 1.0, 0.6666,...",0.666648
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -3-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666, 1.0, 0....",0.694764
Sidney Bechet - Summertime.mid,16,69,4,2,69,4,4,0,Weimar DB,Sidney Bechet - Summertime -4-.mid,2,B- major,Am,2,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0.690039


In [209]:
data = pd.read_csv('../data/split_melody_data/v1.2/Weimar DB/Sidney Bechet - Summertime -1-.csv')

In [210]:
data

,Unnamed: 0,pitch,pitch_class,raw_ticks,quant_ticks,raw_duration,quant_duration,offset,measure,chord_name,chord_root,chord_bass,chord_notes,ticks,duration
0,0,68,Ab,0.244246,12,0.025992,1,12,0,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",12,1
1,1,69,A,0.293552,14,0.031746,2,14,0,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",14,2
2,2,68,Ab,0.377778,18,0.031746,2,18,0,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",18,2
3,3,69,A,0.420833,20,0.060714,3,20,0,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",20,3
4,4,68,Ab,0.498909,24,0.060714,3,24,0,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,69,72,C,14.570337,699,0.309325,15,27,14,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",699,15
70,70,74,D,14.888294,715,0.150298,7,43,14,Amin7,A,A,"['A', 'A', 'C', 'E', 'G']",715,7
71,71,69,A,15.038591,722,0.089583,4,2,15,Bm7b5,B,B,"['B', 'B', 'D', 'F', 'A']",722,4
72,72,68,Ab,15.165675,728,0.118452,6,8,15,Bm7b5,B,B,"['B', 'B', 'D', 'F', 'A']",728,6
